In [1]:
import gym_FPS
import gym
import random
import threading
from gym_FPS.utils import *
from gym_FPS.control import *
from gym_FPS.client2 import Client
from time import *
import numpy as np

env=gym.make('FPSDemo-v3')
env.set_env('10.212.47.201',5123,client_DEBUG=False,env_DEBUG=False)
#env.set_env(client_DEBUG=False,env_DEBUG=False)
env.new_episode()
#env.playerai()
sleep(1)
team_action = 5
team_action_flag = False

sa=[]
sb=[]
sct=[]
#侦察队目标
dest_pos=[[-228.1,-1,94],[-90.6,-1,-84.7],[-90.6,-1,-84.7]]
dest_pos2=[]
curocean_pos= [-216.5,-0.70,-0.8]
dest_pos_t1=[[-208.1,-1,98.6],[-131.3,12.91,112.0],[-154.1,-1,48.9],[-100.6,-1,80.1],[-58.6,-1,28.1]]
dest_pos_t2=[[-148.7,-1,-70.8],[-159.2,-1,-2.4],[-154.1,-1,48.9],[-100.6,-1,80.1],[-58.6,-1,28.1]]

In [2]:
def receive_action():
    global team_action
    global team_action_flag
    #client = Client(SEVERIP='10.180.11.122',SERVERPORT=10243,DEBUG=False)
    client = Client(SERVERPORT=10243,DEBUG=False)
    fff=0
    while True:
        msg = client.receive()
        if msg.find(':')>-1:
            s=msg.split(':')
            if s[0]=='light':
                env.select_obj(int(s[1]))
                continue
            if s[0]=='ui':
                env.add_ui_prompt(s[1])
                print('ui_prompt  %s'%s[1])
            if s[1].find('a')>-1:
                if fff==0:
                    sa.append(int(s[0]))
                    env.create_team(-1,[sa[-1]],3)
                    sct.append(Scout(sa[-1], env, "Pioneer", 'team1', [dest_pos[0]], dest_pos_t1))
                    dest_pos2.append(dest_pos[0])
                    threading.Thread(target=sct[-1].run).start()
                    threading.Thread(target=sct1_check,args=(sa[-1],)).start()
                    msg=str(sa[0])+"号侦察兵正前往目标哨点"
                    env.add_chat(msg=msg,obj_id=sa[0],close_time=5)
                    env.select_obj(int(s[0]),0)
                    fff=1
                elif fff<3:
                    sb.append(int(s[0]))
                    env.create_team(-1,[sb[-1]],3)
                    sct.append(Scout(sb[-1], env, "Pioneer", 'team2', [dest_pos[1]], dest_pos_t2)) 
                    dest_pos2.append(dest_pos[1])          
                    threading.Thread(target=sct[-1].run).start()
                    threading.Thread(target=sct2_check,args=(sb[-1],)).start()
                    msg=str(sb[-1])+"号侦察兵正前往目标哨点"
                    env.add_chat(msg=msg,obj_id=sb[-1],close_time=5)
                    env.select_obj(int(s[0]),0)
                    fff+=1
            '''
            else:
                sb.append(int(s[0]))
                env.create_team(-1,[sb[-1]],3)
                sct.append(Scout(sb[-1], env, "Pioneer", 'team2', [dest_pos[1]], dest_pos_t2)) 
                dest_pos2.append(dest_pos[1])          
                threading.Thread(target=sct[-1].run).start()
                threading.Thread(target=sct2_check,args=(sb[-1],)).start()
                msg=str(sb[-1])+"号侦察兵正前往目标哨点"
                env.add_chat(msg=msg,obj_id=sb[-1],close_time=5)
                env.select_obj(int(s[0]),0)
            '''
            continue
        team_action = int(msg)
        print('receive team_acction:%d'%team_action)
        env.add_ui_prompt('ok')
        team_action_flag = True
        
        if team_action==1:
            team1msg='改变成警戒队型'
        elif team_action==2:
            team1msg='改变成护卫队型'
        elif team_action==3:
            team1msg='改变成进攻队型'
        else:
            team1msg='改变成进攻队型'
        env.add_chat(msg=team1msg,obj_id=0,close_time=5)

In [3]:
def auto_team_action():
    global team_action
    global team_action_flag
    state = 1
    while True:
        enemy_nearby = env.get_enemy_nearby()
        if len(enemy_nearby) > 0 and state == 0:
            print('auto:3')
            print(env.states[0]['POSITION'])          
            team1seg1='主队在行进过程中进攻敌人，成进攻队型'
            team_action = 3
            team_action_flag = True
            state = 1
            env.add_chat(msg=team1seg1,obj_id=0,close_time=5)
        elif len(enemy_nearby) == 0 and state == 1:
            print('auto:1')
            team_action = 1
            team_action_flag = True
            state = 0
            team1seg2='主队在行进过程中已经消灭敌人，成护卫队型'
            env.add_chat(msg=team1seg2,obj_id=0,close_time=5)   
        sleep(0.5)
    

In [4]:
def get_action():
    global team_action
    global team_action_flag
    count = 0;
    while True:
        sleep(0.01)
        if team_action_flag:
            print('action:%d' % team_action)
            team_action_flag = False
            count = 0
        if count % 100 == 0 and (team_action == 1 or team_action == 5 or count == 0):
            print('step:%d'%team_action)
            env.step(team_action)
        count += 1

In [5]:
#场景一：派出侦察员，并消灭敌方哨队
player_list=env.units
player_num=len(player_list)
comradeteam=[]
enemyteam=[]
searchteam_first=[]
searchteam_second=[]
alertteam=[]
towerteam=[]

In [6]:
##敌我分队初始化
for player_id,player_name in player_list.items():
    if player_name.find('队友')>=0:
        comradeteam.append(player_id)
    elif player_id ==0:
        comradeteam.append(player_id)
    elif player_name.find('巡逻')>=0:
        if int(player_name[-1])<=5:
            searchteam_first.append(player_id)
        else:
            searchteam_second.append(player_id)
    elif player_name.find('驻守')>=0:
        alertteam.append(player_id)
    else:
        towerteam.append(player_id)

In [7]:
def sct1_check(uid):
    while True:
        sleep(0.5)
        pos=env.states[uid]['POSITION']
        if pos[2]>50:
            env.add_observer([-210,-1,96],80)
            env.add_observer([-220,-1,50],80)
            break
    while True:
        sleep(0.5)
        pos=env.states[uid]['POSITION']
        if pos[2]>87:
            env.add_observer([-170,-1,96],80)
            env.add_observer([-130,-1,96],80)
            break

In [8]:
def sct2_check(uid):
    while True:
        sleep(0.5)
        pos=env.states[uid]['POSITION']
        if pos[0]>-190:
            env.add_observer([-185,-1,-30],80)
            break
    while True:
        sleep(0.5)
        pos=env.states[uid]['POSITION']
        if pos[0]>-95:
            env.add_observer([-90,-1,-80],80)
            env.add_observer([-90,-1,-30],80)
            break

In [9]:
def hold_position(objid_list):
    curpos=env.get_pos()
    for eid in objid_list:
        if eid in env.units.keys():
            env.move(destPos=curpos[eid],objid_list=[eid],)

In [10]:
def team1_check():
    eid=[]
    for key, value in env.units.items():
        if value in ['驻守18','驻守19','驻守20','驻守21']:
            eid.append(key)
            
    hold_position(eid)
    '''
    while True:
        sleep(0.5)
        pos0=env.states[0]['POSITION']
        if pos0[2] > 45 and pos0[0] > -215:
            break
    print('pos0:',pos0)
    '''  
    global team_action, team_action_flag
    while team_action!=2:
        sleep(0.5)
    sleep(2)
    team_action=2
    team_action_flag=True
    sleep(1)
    env.can_attack_move(eid, destPos=[-210,-1,50],walkType='run')
    print('eid:',eid)
    sleep(5)
    while True:
        if len(set(eid) - set(env.units.keys())) == 0:
            break
        sleep(0.5)
    print(env.units.keys())
    t3 = threading.Thread(target=auto_team_action)
    t3.start()
    print('t3 start')
    
    
    
    while True:
        sleep(1)
        pos0 = env.states[0]['POSITION']
        if pos0[2] > 65:
            break
            
    env.add_obj('地道兵1', True, [-210,-1,50], -1, -1, weapon='ak47')
    env.add_obj('地道兵2', True, [-211,-1,49], -1, -1, weapon='ak47')
    env.add_obj('地道兵3', True, [-209,-1,49], -1, -1, weapon='ak47')
            
    while True:
        sleep(1)
        pos0=env.states[0]['POSITION']
        if pos0[2]>80:
            break
    env.create_team(0, sa, 1)
    global path_id1
    env.remove_pathline(path_id1)
    path_id2 = env.draw_pathline([dest_pos_t1[0], dest_pos_t1[1]])
    print('creat team')
       
    while True:
        sleep(1)
        specialname='站岗1'
        curexistplayer=env.units
        curexistnames=list(curexistplayer.values())
        if specialname not in curexistnames:
            break
    env.register('sys_team1',True)
    env.remove_pathline(path_id2)
    path_id3 = env.draw_pathline([dest_pos_t1[1], dest_pos_t1[2]])
    team1msg='主队将攻击第一据点，请分队支援'
    env.add_chat(msg=team1msg,obj_id=0,close_time=5)
    print('register sys_team1')
    
    while True:
        sleep(1)
        if env.getVal('clear1'):
            break
    team1msg='已经消灭第一据点，将前往下个地点集合'
    env.add_chat(msg=team1msg,obj_id=0,close_time=5)
    env.remove_pathline(path_id3)
    path_id4 = env.draw_pathline([dest_pos_t1[2], dest_pos_t1[3]])
    
    while True:
        sleep(1)
        pos0=env.states[0]['POSITION']
        if get_dis(pos0, dest_pos_t1[-2], False)<10:
            break
    env.register('ready_team1',True)
    env.remove_pathline(path_id4)
    
    while True:
        sleep(1)
        if env.getVal('ready_team2'):
            break
    team2msg='已经集合完毕，将进攻第二据点'
    env.add_chat(msg=team2msg,obj_id=0,close_time=5)
    path_id5 = env.draw_pathline([dest_pos_t1[3], dest_pos_t1[4]])
    print('register ready_team1')

<ipython-input-10-3dabe58d2568>:19: SyntaxWarning: name 'team_action' is used prior to global declaration
  global team_action, team_action_flag


In [11]:
def team1_register():
    while True:
        sleep(1)
        pos0=env.units[0]['POSITION']
        if get_dis(pos0,dest_pos_t2[-4])<10:
            break
    env.register('sys_team1',True)
    env.remove_pathline(path_id2)
    path_id3 = env.draw_pathline([dest_pos_t1[1], dest_pos_t1[2]])
    team1msg='主队将攻击第一据点，请分队先行'
    env.add_chat(msg=team1msg,obj_id=0,close_time=5)
    print('register sys_team1')
    
    while True:
        sleep(1)
        if env.getVal('clear1'):
            break
    team1msg='已经消灭第一据点，将前往下个地点集合'
    env.add_chat(msg=team1msg,obj_id=0,close_time=5)
    env.remove_pathline(path_id3)
    path_id4 = env.draw_pathline([dest_pos_t1[2], dest_pos_t1[3]])
    
    while True:
        sleep(1)
        pos0=env.states[0]['POSITION']
        if get_dis(pos0, dest_pos_t1[-2], False)<10:
            break
    env.register('ready_team1',True)
    env.remove_pathline(path_id4)
    
    while True:
        sleep(1)
        if env.getVal('ready_team2'):
            break
    team2msg='已经集合完毕，将进攻第二据点'
    env.add_chat(msg=team2msg,obj_id=0,close_time=5)
    path_id5 = env.draw_pathline([dest_pos_t1[3], dest_pos_t1[4]])
    print('register ready_team1')

In [12]:
env.register("wait_team1",0)
env.register("wait_team2",0)
env.register('sys_team1',False)
env.register('sys_team2',False)

In [13]:
t1=threading.Thread(target=receive_action)
t1.start()
#t0=threading.Thread(target=team1_register)
#t0.start()

In [14]:
##侦察员出发
env.set_task('派遣三位侦察兵前往侦察哨点')
sctmark=[]
sctmark.append(env.add_map_mark(pos=dest_pos[0]))
sctmark.append(env.add_map_mark(pos=dest_pos[1]))
sctmark.append(env.add_map_mark(pos=dest_pos[2]))

search_num=3
comradeteam.sort()
while len(sa)<1 or len(sb)<2:
    sleep(0.2)
    

comrade_search=sa+sb


In [15]:
#分头行动！老大带人去打老窝，小队负责阻击敌方哨队！
one_attackteam=[]#主队
second_attackteam=[]

rest_playerlist=env.units
for player_id,player_name in rest_playerlist.items():
    if player_id not in comrade_search:
        if player_name.find('队友')>=0:
            if len(second_attackteam) < 5:
                second_attackteam.append(player_id)
            else:
                one_attackteam.append(player_id)
        elif player_id ==0:
            one_attackteam.append(player_id)
print("team1:",len(one_attackteam),", team2:",len(second_attackteam),", our_scouts:",len(comrade_search))

one_attackteam.sort()
second_attackteam.sort()
print(one_attackteam)
print(second_attackteam)

if 0 in one_attackteam:
    env.create_team(0,one_attackteam,1)
    env.create_team(second_attackteam[0],second_attackteam,2)
else:
    env.create_team(one_attackteam[0],one_attackteam,2)
    env.create_team(0,second_attackteam,1)

team1: 5 , team2: 5 , our_scouts: 3
[0, 7, 8, 9, 11]
[1, 2, 3, 4, 5]


In [16]:
goned=[False] * search_num
while True:
    for j in range(search_num):
        if goned[j]==False and sct[j].arrive(dest_pos2[j]):
            goned[j]=True
            msg=str(comrade_search[j])+"号侦察兵到达目标哨点"
            env.add_chat(msg=msg,obj_id=comrade_search[j],close_time=5)
            env.remove_map_mark(sctmark[j])
    #print(goned)
    if False in goned:
        sleep(1)
    else:
        break
        
env.add_map_mark(pos=dest_pos_t1[-3],marktype='focus')

ARRIVEDDDD [-228.1, -1, 94]
ARRIVEDDDD [-90.6, -1, -84.7]
ARRIVEDDDD [-90.6, -1, -84.7]


4

In [17]:
t2=threading.Thread(target=get_action)
t2.start()
#t3=threading.Thread(target=auto_team_action)
#t3.start()
t4=threading.Thread(target=team1_check)
t4.start()

#侦察队出发
for i in range(search_num):
    sct = Scout(comrade_search[i], env, "Pioneer", scout_team[i], [dest_pos[i]], \
                scout_follow[i])
    task_s = threading.Thread(target=sct.run, args=())
    task_s.start()
#    env.move(dest_pos[i],objid_list=[comrade_search[i]],walkType='run')

In [18]:
env.register("wait_team1",0)
env.register("wait_team2",0)
env.register("team1", None)
env.register("team2", None)
env.register("ready_team1", False)
env.register("ready_team2", False)
env.register("clear1",False)
env.register("clear2",False)


step:5
receive action:5
move_follow


In [19]:
#第一战！主队去迂回，分队挡住

sleep(3)

task_t2 = TeamWork("team2",env, second_attackteam, dest_pos_t2)
task_thread2 = threading.Thread(target=task_t2.run, args=())
task_thread2.start()

step:5
receive action:5
move_follow
step:5
receive action:5
move_follow
runit! team2 0


In [20]:
#被敌方发现了！
env.can_attack_move(objid_list=searchteam_first,destPos=curocean_pos)


env.set_task('攻击第一据点')
team1string="主队将采取大迂回，进攻敌人"
team2string="本小队挡住敌人，请主队采取迂回"
env.add_chat(msg=team2string,obj_id=second_attackteam[0],close_time=5)
env.add_chat(msg=team1string,obj_id=0,close_time=5)
path_id1=env.draw_pathline([[-223.0,-1,25.0], [-203,-1,95]])

moniter = Moniter(env)
moniter_task = threading.Thread(target=moniter.run, args=())
moniter_task.start()

we get monitor


In [21]:
1

step:5
receive action:5
move_follow


1